In [2]:
# including the project directory to the notebook level
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
#importing modules
from dataprocess import dataprocessor as dp
from nn_source import models as mp

Using TensorFlow backend.
/data/nauga/SmartBuildings/venvs/sbvenv1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/nauga/SmartBuildings/venvs/sbvenv1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/nauga/SmartBuildings/venvs/sbvenv1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/d

In [4]:
# read the pickled file for ahu data
ahudata = dp.readfile('../data/processed/ahu1energy.pkl')

# return pickled df
ahu = ahudata.return_df(processmethods=['file2df'])

# read the pickled file for ghi data
ghidata = dp.readfile('../data/processed/ghi.pkl')

# return pickled df
ghi = ghidata.return_df(processmethods=['file2df'])

In [5]:
# selecting only required columns and rearranging them
ahu = ahu[[
    'AHU_1 outdoorAirTemp', 'WeatherDataProfile humidity',
    'AHU_1 supplyAirTemp', 'HW_BTU_METER currentKbtuDeltaReading',
    'CHW_BTU_METER currentKbtuDeltaReading'
]]

# renaming columns
ahu.columns = ['oat', 'orh', 'sat', 'hwe', 'cwe']

# Total energy is sum of heating and coling
ahu['totale'] = ahu['hwe'] + ahu['cwe']
# dropping heating and cooling energy columns
ahu = ahu.drop(columns=['hwe', 'cwe'])

In [6]:
# merging ahu and ghi data
df = dp.merge_df_columns([ahu, ghi])

# rearranging columns
df = df[['oat', 'orh', 'sat', 'Ghi', 'totale']]

In [7]:
dflist = dp.df2dflist(df, subsequence=True, period=1, days=7, hours=0)

In [8]:
X_train, X_test, y_train, y_test, X_scaler, y_scaler = dp.df2arrays(
    dflist[0],
    predictorcols=['oat', 'orh', 'sat', 'Ghi'],
    outputcols=['totale'],
    scaling=True,
    reshaping=True)

In [9]:
for i in [X_train, X_test, y_train, y_test]:
    print(i.shape)

(1511, 1, 4)
(504, 1, 4)
(1511, 1, 1)
(504, 1, 1)


In [10]:
# create the results directory
os.mkdir('../results/lstmtrain/')

In [11]:
#Instantiate learner model
model = mp.lstm_model('../results/lstmtrain/',
                      inputdim=4,
                      outputdim=1,
                      period=1)

# Desing model architecture
model.design_model(lstmhiddenlayers=[64, 64], densehiddenlayers=[32])

In [12]:
model.show_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 1, 4)              0         
_________________________________________________________________
reshape_layer (Reshape)      (None, 4)                 0         
_________________________________________________________________
input_repeater (RepeatVector (None, 1, 4)              0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 64)             17664     
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 64)             33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1, 32)             2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1, 1)              33        
Total para

In [13]:
model.model_callbacks()

In [14]:
model.train_model(X_train, y_train, X_test, y_test)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1511 samples, validate on 504 samples
Epoch 1/100
 - 2s - loss: 11.8625 - val_loss: 6.1256
Epoch 2/100
 - 0s - loss: 3.4761 - val_loss: 1.5506
Epoch 3/100
 - 0s - loss: 0.8201 - val_loss: 0.3313
Epoch 4/100
 - 0s - loss: 0.1890 - val_loss: 0.0929
Epoch 5/100
 - 0s - loss: 0.0756 - val_loss: 0.0562
Epoch 6/100
 - 0s - loss: 0.0601 - val_loss: 0.0528
Epoch 7/100
 - 0s - loss: 0.0582 - val_loss: 0.0518
Epoch 8/100
 - 1s - loss: 0.0579 - val_loss: 0.0517
Epoch 9/100
 - 0s - loss: 0.0580 - val_loss: 0.0519
Epoch 10/100
 - 0s - loss: 0.0577 - val_loss: 0.0517
Epoch 11/100
 - 0s - loss: 0.0279 - val_loss: 0.0153
Epoch 12/100
 - 0s - loss: 0.0196 - val_loss: 0.0152
Epoch 13/100
 - 1s - loss: 0.0194 - val_loss: 0.0154
Epoch 14/100
 - 0s - loss: 0.0193 - val_loss: 0.0156
Epoch 15/100
 - 0s - loss: 0.0163 - val_loss: 0.0119
Epoch 16/100
 - 0s - loss: 0.0156 - val_loss: 0.0119
Epoch 17/100
 - 0

In [15]:
preds_train, preds_test = model.evaluate_model(X_train,
                                                         y_train,
                                                         X_test,
                                                         y_test,
                                                         y_scaler,
                                                         saveplot=True)